In [1]:
import pip

package_names = ['lightgbm', 'awswrangler', 'sagemaker']
pip.main(['install'] + package_names + ['--upgrade'])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


0

In [2]:
# Nativos
from dateutil.relativedelta import relativedelta
from time import gmtime, strftime
from datetime import datetime
import random as rn
import joblib
import json
import sys
import os
import gc

#nube
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker import get_execution_role
import awswrangler as wr
import sagemaker
import boto3

#calculo
import pandas as pd
import numpy as np
import scipy

#grafico
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid")

#Interacciones con output
import warnings
warnings.filterwarnings("ignore")
# warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

gc.collect()
# MODELS
from lightgbm import LGBMClassifier

#import scorecardpy as sc

SEED = 29082013
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
rn.seed(SEED)


from setup import *

NumExpr defaulting to 4 threads.
////////////////////////////////////////////////// 

path_container_input:  /opt/ml/processing/input
path_container_output:  /opt/ml/processing/output
path_container_utils:  /opt/ml/processing/input/utils
path_container_universo:  /opt/ml/processing/input/HM_UNIVERSO_PROPENSION_DESEMBOLSO_BPE


### CONFIG

In [3]:
print("bucket: ", bucket)
print("region: ", region)
print("now: ", now)

bucket:  sagemaker-us-east-1-058528764918
region:  us-east-1
now:  2021-09-22 14:24:27.921342


In [4]:
print('uri_universo_athena: ', uri_universo_athena)
print('uri_output: ', uri_output)
print('uri_code: ', uri_code)

print("")
print("/"*50, "\n")

print('path_container_input: ', path_container_input)
print('path_container_output: ', path_container_output)
print('path_container_utils: ', path_container_utils)
print('path_container_universo: ', path_container_universo)

uri_universo_athena:  s3://sagemaker-us-east-1-058528764918/vpc/propension/athena/HM_UNIVERSO_PROPENSION_DESEMBOLSO_BPE
uri_output:  s3://sagemaker-us-east-1-058528764918/vpc/propension/output3
uri_code:  s3://sagemaker-us-east-1-058528764918/vpc/propension/code

////////////////////////////////////////////////// 

path_container_input:  /opt/ml/processing/input
path_container_output:  /opt/ml/processing/output
path_container_utils:  /opt/ml/processing/input/utils
path_container_universo:  /opt/ml/processing/input/HM_UNIVERSO_PROPENSION_DESEMBOLSO_BPE


### SETEADO DE UTILITARIO

In [5]:
print(path_instancia_util)
print(prefix_utils)

/home/ec2-user/SageMaker/DESEMBOLSO_7/UTILITARIO_CODE/utils.py
vpc/propension/code/utils.py


In [6]:
s3.Bucket(bucket).upload_file(path_instancia_util, prefix_utils)

### DESACOPLADO DIRECTO S3

In [7]:
processing_tags = [{'Key': 'cost-center', 'Value': 'TF2WorkflowProcessing'}]

sklearn_processor_a = SKLearnProcessor(
    framework_version='0.23-1',
    role=get_execution_role(),
    instance_type='ml.m5.12xlarge',
    instance_count=1,
    tags=processing_tags
)

Same images used for training and inference. Defaulting to image scope: inference.
Defaulting to only available Python version: py3


In [ ]:
%%time
processing_job_name = "propension-workflow-{}".format(strftime("%Y-%m-%d-%H-%M-%S", gmtime()))

sklearn_processor_a.run(
    code='feature_partition.py',  #'/opt/ml/processing/codigo_proceso/' , '/opt/ml/processing/input/code'
    job_name=processing_job_name,
    inputs=[
        ProcessingInput(
            source=uri_universo_athena,
            destination=path_container_universo,
            s3_data_distribution_type='ShardedByS3Key'
        ),
        ProcessingInput(
            source=uri_code,
            destination=path_container_utils,
            s3_data_distribution_type='ShardedByS3Key'
        ),
    ],
    outputs=[
        ProcessingOutput(
            output_name='result',
            destination=uri_output,
            source=path_container_output
        )
    ]
)

preprocessing_job_description = sklearn_processor_a.jobs[-1].describe()


Job Name:  propension-workflow-2021-09-22-14-24-28
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-058528764918/vpc/propension/athena/HM_UNIVERSO_PROPENSION_DESEMBOLSO_BPE', 'LocalPath': '/opt/ml/processing/input/HM_UNIVERSO_PROPENSION_DESEMBOLSO_BPE', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'ShardedByS3Key', 'S3CompressionType': 'None'}}, {'InputName': 'input-2', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-058528764918/vpc/propension/code', 'LocalPath': '/opt/ml/processing/input/utils', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'ShardedByS3Key', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-058528764918/propension-workflow-2021-09-22-14-24-28/input/code/feature_partition.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'Fil

In [ ]:
#    arguments=["--ultimo_mes", ultimo_mes]

### COSECHA ESTRATEGIA

In [ ]:
lista = ['202008', '202009', '2020010', '202011', '202012', '202101', '202102', '202103', '202104', '202105', '202106', '202107']

In [ ]:
print(lista[:-5])

In [ ]:
lista[-6:-4]

In [ ]:
lista[-4:]